# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [2]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [8]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'gemma3:4b'

In [4]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")


API key looks good so far


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
# Get gpt-4o-mini to answer, with streaming

openai = OpenAI()

stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "user", "content": question}
      ],
    stream=True
)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)



The line of code you've shared uses Python's `yield from` statement combined with a set comprehension to generate a series of values. Let's break it down:

1. **`{... for book in books if book.get("author")}`**: This is a set comprehension that iterates over a collection named `books`. For each `book` in `books`, it checks whether the key `"author"` exists and has a truthy value (using `book.get("author")`). If the key exists and the value is not `None`, `False`, or any other value that evaluates to `False`, it adds that author's name to a set. The use of set comprehension ensures that the resultant collection of authors will not contain duplicates.

2. **`yield from ...`**: The `yield from` statement is used in Python generators. It allows one generator to delegate part of its operations to another iterable. When you yield from an iterable, it produces each value from that iterable one at a time to the caller, while also maintaining the state of the generator. Essentially, this allows for more organized code when working with generators.

### Purpose of the Combined Elements
Given the way the code is structured, here's what the entire line does:

- It collects unique authors from a list of `books`, where each `book` is expected to be a dictionary that might contain an `"author"` key.
- It yields each unique author one by one when the generator is called.

### Example
If `books` looks like this:
python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},  # Duplicate author
    {"title": "Book 4"}  # No author
]


The code would effectively yield:
- "Author A"
- "Author B"

### Why Use This Code?
1. **Efficiency**: Using a set comprehension ensures that duplicate authors are automatically handled.
2. **Readability**: The `yield from` syntax allows the creator of the generator to keep the code clean and clear, especially when yielding multiple values.
3. **Lazy Evaluation**: This generator approach allows for values to be processed one at a time, which is more memory efficient than generating a complete list of authors upfront, especially if `books` is large.

In summary, the code is designed to yield each unique author from a list of book dictionaries, skipping any book that does not have an author defined, and it does so in a way that is both memory and computation efficient.

In [ ]:
# Get Llama 3.2 to answer

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

stream = openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[
        {"role": "user", "content": question}
      ],
    stream=True
)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)


Okay, let's break down this Python code snippet:

python
yield from {book.get("author") for book in books if book.get("author")}


**What the Code Does**

This code is a generator expression that uses a `yield from` statement to produce a sequence of author names.  Here's a step-by-step explanation:

1. **`books`:** This variable is assumed to be a list or iterable containing dictionaries. Each dictionary in the list represents a "book."  Each book dictionary is expected to have a key named "author."

2. **`for book in books`:** This loop iterates through each dictionary (`book`) in the `books` collection.

3. **`if book.get("author")`:**
   - `book.get("author")`: This is a safe way to access the value associated with the key "author" in the `book` dictionary.  The `.get()` method is used instead of direct dictionary access (`book["author"]`) because `book["author"]` would raise a `KeyError` if a book dictionary *doesn't* have an "author" key.  `.get()` returns `None` if the key is missing.
   - `if book.get("author")`: This condition is effectively a boolean